# AUTOGEN MULTI-AGENT ORCHESTRATION

AutoGen's multi-agent orchestration enables seamless collaboration between AI agents, allowing them to autonomously delegate tasks, retrieve information, and execute complex workflows. By leveraging structured communication, tool integration, and intelligent decision-making, AutoGen facilitates scalable AI-driven applications across domains like research, automation, and customer support. With flexible orchestration models such as RoundRobin, SelectorGroupChat, and Swarm, developers can design adaptive multi-agent ecosystems that efficiently coordinate and optimize task execution.

Using multiple agents in AI Automation have the following benefits:
- **Specialization:** Instead of one massive model trying to handle everything (translation, coding, web-surfing, etc.), you can compose smaller, specialized agents.
- **Parallelism & Modularity:** Multiple agents can divide tasks or work in parallel without stepping on each other’s toes.
- **Scalable Expertise:** You can plug in new agents (like a VideoSurfer, FileSurfer, or SQLQueryAgent) without rewriting existing logic.


New requirements to manage the complexity of using multiple agents...
- **Orchestration:** If multiple agents can respond at once, you need a coordinating mechanism (how do they know who speaks next? do they pass tasks around?). AutoGen provides RoundRobinGroupChat, SelectorGroupChat, Swarm, and MagenticOneGroupChat for this orchestration.
- **Shared Context & State:** Agents must see the most recent conversation state and outputs of other agents. AutoGen’s team concept ensures agents share a single conversation thread or message buffer.
- **Tool Integration:** Each agent might call external APIs. If they conflict, you need to define a tool schema or allow parallel/discrete usage. In AutoGen, you can pass different tools or function-call endpoints to each agent. That way, the finance agent might have a currency API tool, while a web-search agent has a search tool.
- **Memory & Persistence:** Long-running multi-agent applications need a memory store to recall user preferences, partial results, or conversation states across steps. AutoGen’s Memory interface (e.g., ListMemory, vector-based memory) can store facts so that all agents can retrieve relevant knowledge.-

Below is a minimal multi-agent orchestration example using Azure OpenAI. We create two agents—one with multiple tools, and a second “critic” agent—then place them in a RoundRobinGroupChat so they can take turns responding. This setup demonstrates how to integrate Azure OpenAI, load environment variables, define tool functions, and orchestrate more than one agent in a conversation.

In [2]:
import os
import asyncio
from dotenv import load_dotenv

# Agents, team, console UI
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Azure OpenAI client
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# 1) Load Azure environment variables
load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "")

# 2) Create Azure OpenAI Client
model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o",
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

# 3) Define two agents
quantum_explainer = AssistantAgent(
    name="QuantumExplainer",
    model_client=model_client,
    system_message="You are a knowledgeable assistant who provides clear explanations of quantum physics principles."
)

critic_agent = AssistantAgent(
    name="Critic",
    model_client=model_client,
    system_message="You specialize in providing constructive critiques and clarifications about scientific explanations."
)

# 4) RoundRobinGroupChat: agents take turns automatically
team = RoundRobinGroupChat(
    participants=[quantum_explainer, critic_agent]
)

# 5) Give them a shared task in quantum physics (Jupyter-friendly)
await Console(team.run_stream(task="Explain the basics of quantum entanglement in simple terms."))


---------- user ----------
Explain the basics of quantum entanglement in simple terms.
---------- QuantumExplainer ----------
Quantum entanglement is a fascinating phenomenon that occurs when two or more particles become correlated in such a way that the state of one particle instantly influences the state of the other, no matter how far apart they are. Here’s a simplified explanation:

Imagine you have two entangled particles, like a pair of shoes: one left shoe and one right shoe. If you separate these shoes and send one to New York and the other to Tokyo, as soon as you check the shoe in New York and see that it's the left one, you instantly know that the shoe in Tokyo is the right one.

In the quantum world, particles like electrons or photons can become entangled through certain interactions. Once entangled, the measurement of a specific property, such as spin or polarization, on one particle will determine the outcome of the same measurement on its entangled partner, regardless o

Error processing publish message for Critic/43aa14a8-cdf1-45c3-9d13-17b8f9e33929
Traceback (most recent call last):
  File "/Users/ozgurguler/Developer/Projects/htp_fintech_workshop_2502/venv/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 505, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ozgurguler/Developer/Projects/htp_fintech_workshop_2502/venv/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ozgurguler/Developer/Projects/htp_fintech_workshop_2502/venv/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 48, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ozgurguler/Developer/Projects/htp_fintech_worksh

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Explain the basics of quantum entanglement in simple terms.', type='TextMessage'), TextMessage(source='QuantumExplainer', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=260), content='Quantum entanglement is a fascinating phenomenon that occurs when two or more particles become correlated in such a way that the state of one particle instantly influences the state of the other, no matter how far apart they are. Here’s a simplified explanation:\n\nImagine you have two entangled particles, like a pair of shoes: one left shoe and one right shoe. If you separate these shoes and send one to New York and the other to Tokyo, as soon as you check the shoe in New York and see that it\'s the left one, you instantly know that the shoe in Tokyo is the right one.\n\nIn the quantum world, particles like electrons or photons can become entangled through certain interactions. Once entangled, the measurement of a 